In [1]:
import os.path
import keras.applications.mobilenet
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
import PIL.Image
import matplotlib.pyplot as plt

Using TensorFlow backend.
/home/stanislav/Documents/notebooks/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/stanislav/Documents/notebooks/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/stanislav/Documents/notebooks/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/h

In [2]:
DATA_ROOT = os.path.expanduser("~/Documents/data/")
BATCH_SIZE = 32
STEPS_PER_EPOCH = 1000

The model: two dense layers on top of a mobile Net

In [3]:
def new_model():
    base_model = MobileNet(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    prediction = Dense(1, activation='sigmoid')(x)
    return Model(inputs=base_model.input, outputs=prediction)

In [4]:
def base_layers(model):
    return model.layers[:-3]

In [5]:
def dense_layers(model):
    return model.layers[-2:]

Training

In [ ]:
def train(model, data_gen, directory, trainable_layers=[], params={},
          epochs=4, validation_directory=None, validation_generator=None):

    for layer in model.layers:
        layer.trainable = (layer in trainable_layers)

    gen = data_gen.flow_from_directory(directory,
                                       class_mode='binary',
                                       batch_size=BATCH_SIZE)

    if validation_generator:
        val_gen = val_generator.flow_from_directory(validation_directory,
                                                    class_mode='binary')
    else:
        val_gen = None

    model.compile(**params)
    model.fit_generator(gen,
                        epochs=epochs,
                        steps_per_epoch=STEPS_PER_EPOCH,
                        validation_data=val_gen)

In [7]:
DENSE_TRAIN = {'optimizer': 'rmsprop',
               'loss': 'binary_crossentropy',
               'metrics': ['accuracy']}

In [8]:
BASE_TRAIN = {'optimizer': SGD(lr=0.0001, momentum=0.9),
              'loss': 'binary_crossentropy',
              'metrics': ['accuracy']}

In [9]:
def train1(model, data_gen, directory, epochs=4,
           validation_generator=None, validation_directory=None):
    """Train the dense layers"""
    train(model, data_gen, directory,
          epochs=epochs,
          trainable_layers=dense_layers(model),
          params=DENSE_TRAIN)

In [10]:
def train2(model, data_gen, directory, epochs=4,
           validation_generator=None, validation_directory=None):
    """Train some of the base layers"""
    train(model, data_gen, directory,
          epochs=epochs,
          trainable_layers=base_layers(model)[-16:],
          params=BASE_TRAIN)

In [ ]:
model = new_model()
model.summary()
dg_train = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)
dg_val = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)

In [ ]:
train1(model, dg_train, os.path.join(DATA_ROOT, 'part1'),
       validation_directory=os.path.join(DATA_ROOT, 'part2'),
       validation_generator=dg_val)
train2(model, dg_train, os.path.join(DATA_ROOT, 'part1'),
       validation_generator=dg_val,
       validation_directory=os.path.join(DATA_ROOT, 'part2'))